# 在Bokeh中使用Javascript

在Bokeh中可以使用`CustomJS`调用Javascript程序在浏览器中实现各种计算、显示任务。在编写浏览器端运行的程序时，同此案我们希望浏览器载入缩小化之前的Javascript代码，方便调试。可以在载入Bokeh相关的模块之前，设置环境变量`BOKEH_MINIFIED`为`"false"`。

In [1]:
import os
os.environ["BOKEH_MINIFIED"] = "false"

In [2]:
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, ColorBar, CustomJS, Model
from bokeh.models.mappers import LinearColorMapper
from bokeh.palettes import Viridis, Category10
from bokeh.plotting import figure
from bokeh.layouts import row, column
output_notebook()

Loading BokehJS ...

## 对`ColumnDataSource`中的某列进行转换

`bokeh.transform`中提供的各种转换函数，例如`dodge()`将指定列的值加上某个指定的数值。在下面的例子中，`line2`的Y轴数据被设置为`dodge("y", 1)`。它表示将其对应的`ColumnDataSource`中的Y列偏移`1.0`。`dodge()`的运算不是在Python中执行，而是在浏览器中由Javascript执行。

In [3]:
from bokeh.transform import dodge
x = np.linspace(0, 2*np.pi, 100)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y))
fig = figure(plot_width=400, plot_height=250)
line1 = fig.line("x", "y", source=source)
line2 = fig.line("x", dodge("y", 1), color="red", source=source)
show(fig)

曲线对应的数据在Bokeh内部是使用一个字典描述的。其`'field'`属性指定数据在`ColumnDataSource`中对应的列名，而`'transform'`则指定对该列进行的运算。

In [4]:
props = line2.glyph.properties_with_values()
print(props["x"], props["y"])

{'field': 'x'} {'field': 'y', 'transform': Dodge(id='dbbd0840-8146-45ed-ac62-95e2616d0c82', ...)}


`jitter()`对数据叠加上指定大小的随机噪声，在绘制精度有限的散列图使用它能更好地显示数据的分布情况。

In [5]:
from bokeh.transform import jitter
x, y = np.round(np.random.normal(scale=0.2, size=(2, 500)), 1)
source = ColumnDataSource(data=dict(x=x, y=y))
fig1 = figure(plot_width=250, plot_height=250)
fig1.circle("x", "y", source=source, alpha=0.3)
fig2 = figure(plot_width=250, plot_height=250)
fig2.circle(jitter("x", 0.1), jitter("y", 0.1), source=source, alpha=0.3)
show(row(fig1, fig2))

颜色映射表也是通过`transform`模块中提供的各种数值颜色转换函数实现的。例如下面使用`linear_cmap()`将`z`列数据转换为颜色，调色板采用256色的`Viridis`。

In [6]:
from bokeh.transform import linear_cmap
from bokeh.models import ColorBar
from bokeh.palettes import Viridis

x, y = np.random.normal(scale=0.2, size=(2, 500))
z = (x**2 + y**2)**0.5
source = ColumnDataSource(data=dict(x=x, y=y, z=z))
fig = figure(plot_width=400, plot_height=300)
c = fig.circle("x", "y", fill_color=linear_cmap("z", Viridis[256], z.min(), z.max()), 
           line_color=None, source=source, alpha=1, size=6)
colorbar = ColorBar(color_mapper=c.glyph.fill_color["transform"], label_standoff=12, border_line_color=None, location=(0,0))
fig.add_layout(colorbar, "right")
show(fig)

`fill_color`属性的`"transform"`是一个`LinearColorMapper`对象，上面的程序中将它传递给`ColorBar`的`color_mapper`属性，显示颜色条。

In [7]:
c.glyph.properties_with_values()["fill_color"]

{'field': 'z',
 'transform': LinearColorMapper(id='e518f57e-c3e2-4a2d-bd3a-e8072aeb8d67', ...)}

`linear_cmap()`将连续的数值转换为颜色，而`factor_cmap()`将离散的值转换为颜色。通常这种离散的值由Pandas中的`DataFrame`的分类类型表示。分类类型的数据在`ColumnDataSource`中采用字符串表示，因此`factor_cmap()`还需要第三个参数指定分类字符串的列表，颜色与分类的对应关系由该列表中分类的顺序决定。

In [9]:
from bokeh.transform import factor_cmap
df = pd.read_csv("https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv", dtype={'species':'category'})

source = ColumnDataSource(data=df)
fig = figure(plot_width=400, plot_height=300)
c = fig.circle("sepal_length", "sepal_width", 
               fill_color=factor_cmap("species", Category10[10], df.species.cat.categories), 
               legend="species",
               line_color=None, source=source, alpha=1, size=6)
show(fig)

前面的各种函数都是自动生成带`'transform'`键的字典。下面直接通过`transform()`直接指定`'transform'`键对应的转换对象。下面的例子使用`LinearInterpolator`和`StepInterpolator`对指定的列进行插值运算。`source`的`x`列有10个数据点，而`source2`的`x`列有100个点。使用`LinearInterpolator(x="x", y="y", data=source)`对`source`的数值进行插值。

In [11]:
from bokeh.transform import transform
from bokeh.models.transforms import LinearInterpolator, StepInterpolator

x = np.linspace(0, 2*np.pi, 10)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y))
source2 = ColumnDataSource(data=dict(x=np.linspace(0, 2*np.pi, 100)))
fig = figure(plot_width=400, plot_height=300)
fig.circle("x", "y", source=source)
fig.circle("x", transform("x", LinearInterpolator(x="x", y="y", data=source)), source=source2, size=1, color="red", line_color=None)
fig.circle("x", transform("x", StepInterpolator(x="x", y="y", data=source)), source=source2, size=1, color="green", line_color=None)
show(fig)

`CustomJSTransform()`给定的Javascript函数包装为转换类。使用其`from_py_func()`可以将Python函数自动转换为Javascript函数。它有两个参数：对单个数值进行转换的函数，和对一个数组进行转换的函数。这两个函数本身没有参数，函数内部的程序可以通过`x`和`xs`访问传入的数值和数组。下面使用`CustomJSTransform()`创建正弦和余弦转换函数。注意由于计算在Javascript中进行，因此使用`Math.sin()`和`Math.cos()`进行计算。由于生成的Javascript函数不支持对JavaScript的类型化数组进行迭代，因此首先使用`window.Array["from"](xs)`将其转换为一般数组。

In [12]:
from bokeh.models.transforms import CustomJSTransform
x = np.linspace(0, 2*np.pi, 100)
source = ColumnDataSource(data=dict(x=x))
fig = figure(plot_width=400, plot_height=250)

def trans_sin():
    return Math.sin(x)

def vtrans_sin():
    return [Math.sin(x) for x in window.Array["from"](xs)]

transform_sin = CustomJSTransform.from_py_func(trans_sin, vtrans_sin)

def trans_cos():
    return Math.cos(x)

def vtrans_cos():
    return [Math.cos(x) for x in window.Array["from"](xs)]

transform_cos = CustomJSTransform.from_py_func(trans_cos, vtrans_cos)

fig.line("x", transform("x", transform_sin), source=source)
fig.line("x", transform("x", transform_cos), source=source, line_color="red")
show(fig)

所有的转换函数只能针对`ColumnDataSource`对象中的某列数据进行。如果需要使用多列的数据，则可以通过关键字参数将`ColumnDataSource`对象直接传递给转换函数。在下面的`vtrans_func()`中，`source`表示`ColumnDataSource`对象，通过`source.data.x[i]`和`source.data.y[i]`可以获取`x`列与`y`列的第`i`个数值。使用这种方法的缺点是转换对象已经与数据源绑定，无法应用到多个数据源。

In [13]:
x, y = np.random.normal(scale=0.2, size=(2, 500))
source = ColumnDataSource(data=dict(x=x, y=y))
fig = figure(plot_width=400, plot_height=300)

def trans_func(source=source):
    return 0

def vtrans_func(source=source):
    data = source.data
    return [10 * (data.x[i]**2 + data.y[i]**2)**0.5 for i in range(len(data.x))]

c = fig.circle("x", "y", size=transform("x", CustomJSTransform.from_py_func(trans_func, vtrans_func)), 
           line_color=None, source=source, alpha=1)

show(fig)

下面的`composite_transform()`可以将多个转换器合并成一个转换器。

In [14]:
from inspect import Signature, Parameter

def composite_transform(*transforms):
    def trans_func():
        transforms = arguments
        res = x
        for transform in transforms.values():
            res = transform.compute(res)
        return res

    def vtrans_func():
        transforms = arguments
        res = window.Array["from"](xs)
        for transform in transforms.values():
            res = transform.v_compute(res)
        return res
    
    parameters = [Parameter("T{:02d}".format(i), Parameter.POSITIONAL_OR_KEYWORD, default=trans) 
                      for i, trans in enumerate(transforms)]
    trans_func.__signature__ = Signature(parameters=parameters)
    vtrans_func.__signature__ = Signature(parameters=parameters)
    trans = CustomJSTransform.from_py_func(trans_func, vtrans_func)
    return trans

下面使用将`composite_transform()`将两个转换器合并：先用`value_transform`通过`x`和`y`列计算出数值，然后用`cmap_transform`将数值转换为颜色。

In [15]:
x, y = np.random.normal(scale=0.2, size=(2, 500))
source = ColumnDataSource(data=dict(x=x, y=y))
fig = figure(plot_width=400, plot_height=300)

def dummy(source=source):
    return 0

def vtrans_value(source=source):
    data = source.data
    return [(data.x[i]**2 + data.y[i]**2)**0.5 for i in range(len(data.x))]

def vtrans_size():
    return [10 * x for x in window.Array["from"](xs)]

value_transform = CustomJSTransform.from_py_func(dummy, vtrans_value)
mult_transform = CustomJSTransform.from_py_func(dummy, vtrans_size)
cmap_transform = LinearColorMapper(Viridis[256], low=0, high=0.6)
color_transform = composite_transform(value_transform, cmap_transform)
size_transform = composite_transform(value_transform, mult_transform)

c = fig.circle("x", "y", 
               fill_color=transform("x", color_transform), 
               size=transform("x", size_transform),
               line_color=None, source=source, alpha=1)

colorbar = ColorBar(color_mapper=cmap_transform, label_standoff=12, border_line_color=None, location=(0,0))
fig.add_layout(colorbar, "right")
show(fig)

## 视图与过滤器

`ColumnDataSource`是用于保存数据的数据源对象，如果我们希望只显示其中的一部分数据，可以在Python中对数据进行过滤，创建新的`ColumnDataSource`对象，或者使用`CDSView`对象对数据源进行过滤。`CDSView`有两个参数：`source`为数据源，`filters`为过滤器列表。下面使用`GroupFilter`对数据源的`"species"`列进行过滤，值显示`group`指定的分类。

在`Select`的回调函数`on_group_changed()`中修改`group_filter.group`属性，并调用`source.change.emit()`刷新显示。这样用户可以通过界面中的下拉选择框选择需要显示的分类。

In [16]:
from bokeh.models import GroupFilter, CDSView, Select
df = pd.read_csv("https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv", dtype={'species':'category'})
source = ColumnDataSource(data=df)
group_filter = GroupFilter(column_name="species", group="setosa")
view = CDSView(source=source, filters=[group_filter])

fig = figure(plot_width=400, plot_height=300)
c = fig.circle("sepal_length", "sepal_width", 
               line_color=None, source=source, view=view, alpha=1, size=6)

def on_group_changed(filter=group_filter, source=source):
    filter.group = cb_obj.value
    source.change.emit()

select = Select(options=df.species.cat.categories.tolist(),
                callback=CustomJS.from_py_func(on_group_changed))
show(column(select, fig))

和`CustomJSTransform`类似，`CustomJSFilter`可以使用自定义的函数对数据进行过滤。下面使用`func_filter()`选取`source`的`petal_length`列在`slider`指定的范围之内的数据。它返回包含符合条件的下标数组。在`slider`的回调函数中调用`source.change.emit()`触发数据源的更新事件即可重新调用`func_filter`对数据源进行过滤。

In [17]:
from bokeh.models import RangeSlider, CustomJSFilter
source = ColumnDataSource(data=df)
fig = figure(plot_width=400, plot_height=300)
slider = RangeSlider(start=df.petal_length.min(), end=df.petal_length.max(), 
                     value=(df.petal_length.min(), df.petal_length.max()), step=0.1, title="petal length")

def func_filter(source=source, slider=slider):
    data = source.data['petal_length']
    start, end = slider.value
    res = []
    for i in range(len(data)):
        res.append(start <= data[i] <= end)
    return res
    
view = CDSView(source=source, filters=[CustomJSFilter.from_py_func(func_filter)])

c = fig.circle("sepal_length", "sepal_width", 
               fill_color=linear_cmap("petal_length", Viridis[256], df.petal_length.min(), df.petal_length.max()),
               line_color=None, source=source, view=view, alpha=1, size=6)

def on_range_changed(source=source):
    source.change.emit()
    
slider.callback = CustomJS.from_py_func(on_range_changed)    

show(column(slider, fig))